In [20]:
import geopandas as gpd
import pandas as pd
import overpy
import requests
import os
import json
import folium
from folium.plugins import HeatMap
import branca.colormap as cm 


useragent = 'Your website or application name/your contact'
headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '"Google Chrome 80"',
    'Accept': '*/*',
    'Sec-Fetch-Dest': 'empty',
    'User-Agent': useragent,
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://overpass-turbo.eu',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://overpass-turbo.eu/',
    'Accept-Language': '',
    'dnt': '1',
}

COUNTIES_PATH = 'data/CA_Counties/CA_Counties_TIGER2016.shp'
POP_DENSITY_DIR = 'popDensity/'
OSM_QUERY = POP_DENSITY_DIR + 'query.osm'
MAP_DIR = POP_DENSITY_DIR + 'maps/'
PRED_MAP_DIR = POP_DENSITY_DIR + 'pred_maps/'
FLU_BY_REGION_DIR = 'flu_region/'
FLU_BY_DATES_DIR = f"{FLU_BY_REGION_DIR}flu_by_dates"
FLU_BY_DATES_PRED_DIR = f"{FLU_BY_REGION_DIR}pred_by_dates"

assert( os.path.isdir( FLU_BY_REGION_DIR ) )
assert( os.path.isdir( FLU_BY_DATES_DIR ) )
assert( os.path.isdir( FLU_BY_DATES_PRED_DIR ) )

assert( os.path.isfile( COUNTIES_PATH ) )
if not os.path.isdir( POP_DENSITY_DIR ):
    os.mkdir( POP_DENSITY_DIR )
if not os.path.isdir( MAP_DIR ):
    os.mkdir( MAP_DIR )
if not os.path.isdir( PRED_MAP_DIR ):
    os.mkdir( PRED_MAP_DIR )


Get the OSM data for 

In [4]:
counties = gpd.read_file( COUNTIES_PATH )
counties = counties.to_crs({'init': 'epsg:4326'})
bbox = counties.total_bounds
# tighter bounding box than the box above
calBounds = "32.6856199 -114.2028809 34.5246615 -113.9941406 39.0447860 -119.9926758 41.9839943 -119.9487305 41.9676592 -125.5737305 40.2459915 -124.9365234 37.3526928 -124.2993164 32.6578757 -121.3549805 32.6578757 -114.2358398 32.6856199 -114.2028809"


c:\Users\Michael\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [6]:

query = f"""
[out:xml] [timeout:50];
(
    nwr["place"](poly:"{calBounds}" );
);
out body;
"""

data = {
  'data' : query 
}
# call the api to get the query
response = requests.post( 'https://overpass-api.de/api/interpreter', headers=headers, data=data )
# write the response to file for reusing
with open( 'popDensity/query.osm', 'w' ) as f:
  f.write( response.text )
  f.close()
assert( os.path.isfile( OSM_QUERY ) )


In [7]:
# parse the resulting api call
api = overpy.Overpass()
result = api.parse_xml( response.text )

In [8]:
nodes = []
# make the geojson for each of the nodes that have a population.
for i in result.nodes:
  if( i.tags.get('place') != 'state' ):
    if( 'population' in i.tags ):
      pop = i.tags.get( 'population', -1 ) # note pop is a string by default
      if pop != -1:
        pop = int( pop )
        nodes.append( {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [ i.lon, i.lat ],
        },
        "properties": {
          "population" : i.tags.get( 'population', 0 )
        }
});
  
gdfNodes = gpd.GeoDataFrame.from_features([node for node in nodes], crs="EPSG:4326")

In [32]:
def makeMap( caTotalILI, min, max, csvFile, outputDir ):
  output = outputDir + os.path.splitext(csvFile)[0] + ".html"
  # create a folium map centered on the bounding box of cali
  m = folium.Map( location=[ ( bbox[3] + bbox[1] ) / 2, ( bbox[0] + bbox[2] ) / 2 ], zoom_start=7, prefer_canvas=True )

  #color scale
  linear = cm.linear.Reds_09.scale( min, max )
  linear.caption = 'Total Influenza Like Illness (ILI)'
  m.add_child( linear )
  def styleFn( feature ):
      """Function to apply color based on Total_ILI value."""
      Total_ILI = feature['properties']['Total_ILI']
      return {
          'fillColor': linear( Total_ILI ),
          'color': 'black',
          'weight': 2,
          'fillOpacity': .5
      }

  # add the counties shapefile
  geojson = json.loads( caTotalILI.to_json() )
  tooltip = folium.GeoJsonTooltip( fields=['Total_ILI'] )

  # add the points for the heatmap
  data = [ [ point.xy[1][0], point.xy[0][0], int( pop ) ] for point, pop in zip( gdfNodes.geometry, gdfNodes['population'] ) ]

  # create feature group
  featILI = folium.FeatureGroup( name="Total ILI" ).add_to( m )
  featHeat = folium.FeatureGroup( name="Population Density" ).add_to( m )

  #add the total ILI to the feature group
  folium.GeoJson( geojson, tooltip=tooltip, style_function=styleFn ).add_to( featILI )
  # create a HeatMap layer using the population data
  HeatMap( data, overlay=True, control=True ).add_to( featHeat )
  # add the layer control to the map
  folium.LayerControl().add_to( m )

  # display the map
  m.save( output )  # Save the heatmap as an HTML file
  

generate all the file for all the actual csv's

In [ ]:
csvFiles = [ filename for filename in os.listdir( FLU_BY_DATES_DIR ) if filename.endswith( ".csv" ) ]
ca = gpd.read_file( COUNTIES_PATH )

for csvFile in csvFiles:
# if True:
  csvPath = os.path.join( FLU_BY_DATES_DIR, csvFile )
  df = pd.read_csv( csvPath )
  # Calculate the maximum and minimum values in the "Total_ILI" column
  max_value = df["Total_ILI"].max()
  min_value = df["Total_ILI"].min()
  
  #for pred stuff only 
  ca["COUNTYNS"] = ca["COUNTYNS"].astype(int)
  
  #Merge files
  mapf = "COUNTYNS"
  statf = "region"
  df = df[ ['Total_ILI', 'region' ] ]
  # df['region'] = df['region'].astype( str )
  caTotalILI = ca.merge( df, left_on=mapf, right_on=statf )
  caTotalILI = caTotalILI.to_crs( 'epsg:4326' )
  makeMap( caTotalILI, min_value, max_value, csvFile, MAP_DIR )

generate all the csv for the pred maps

In [35]:
csvFiles = [ filename for filename in os.listdir( FLU_BY_DATES_PRED_DIR ) if filename.endswith( ".csv" ) ]
ca = gpd.read_file( COUNTIES_PATH )

for csvFile in csvFiles:
# if True:
  csvPath = os.path.join( FLU_BY_DATES_PRED_DIR, csvFile )
  df = pd.read_csv( csvPath )
  # Calculate the maximum and minimum values in the "Total_ILI" column
  max_value = df["Total_ILI"].max()
  min_value = df["Total_ILI"].min()
  
  #for pred stuff only 
  ca["COUNTYNS"] = ca["COUNTYNS"].astype(int)
  
  #Merge files
  mapf = "COUNTYNS"
  statf = "region"
  df = df[ ['Total_ILI', 'region' ] ]
  # df['region'] = df['region'].astype( str )
  caTotalILI = ca.merge( df, left_on=mapf, right_on=statf )
  caTotalILI = caTotalILI.to_crs( 'epsg:4326' )
  makeMap( caTotalILI, min_value, max_value, csvFile, PRED_MAP_DIR )
